In [1]:
import pprint
from types import SimpleNamespace
from OptiHPLCHandler import EmpowerHandler, EmpowerInstrumentMethod
from typing import List, Optional, Union, Dict

In [2]:
# get api address from .env file
import os
from dotenv import load_dotenv
load_dotenv('../.env_vars')
EMPOWER_API_ADDRESS = os.getenv('EMPOWER_API_ADDRESS')



In [1]:
import sys
sys.path.insert(0, '../')

In [16]:
# Create an instance of the EmpowerHandler class
handler = EmpowerHandler(project="WebAPI_test", address=EMPOWER_API_ADDRESS)

handler.connection.default_get_timeout=300
handler.connection.default_post_timeout=300

# Get the list of methods, select one, and get the method details
with handler:
    method_list = handler.GetMethodList()  # Get the list of instrument methods
    method_name = "20240416_test"
    full_method = handler.GetInstrumentMethod(method_name)

NameError: name 'EmpowerHandler' is not defined

In [2]:
from method_generators.ramp_method import generate_ramp_method
from types import SimpleNamespace
def test_generate_ramp_method():
        method = SimpleNamespace(
            gradient_table=[
                {
                    "Time": "Initial",
                    "Flow": "1",
                    "CompositionA": "95",
                    "CompositionB": "5",
                    "Curve": "Initial",
                },
                {
                    "Time": "10",
                    "Flow": "1",
                    "CompositionA": "10",
                    "CompositionB": "90",
                    "Curve": "6",
                },
                {
                    "Time": "20",
                    "Flow": "1",
                    "CompositionA": "95",
                    "CompositionB": "5",
                    "Curve": "6",
                },
                {
                    "Time": "30",
                    "Flow": "1",
                    "CompositionA": "95",
                    "CompositionB": "5",
                    "Curve": "6",
                },
            ],
            column_temperature=30,
            method_name="test",
        )

        method = generate_ramp_method(
            method, low_flow_rate=0.05, flow_curve=6, ramp_type="rampup"
        )
        print(method.gradient_table)
        expected_method_gradient_table= [
            {
                "Time": "Initial",
                "Flow": "0.05",
                "CompositionA": "95",
                "CompositionB": "5",
                "Curve": "Initial",
            },
            {
                "Time": "10",
                "Flow": "1",
                "CompositionA": "95",
                "CompositionB": "5",
                "Curve": "6",
            },
        ]
        print(expected_method_gradient_table)
        assert method.gradient_table == expected_method_gradient_table
        assert method.column_temperature == 30
        assert method.method_name == "test_ramp"

        method = SimpleNamespace(
            gradient_table=[
                {
                    "Time": "Initial",
                    "Flow": "1",
                    "CompositionA": "95",
                    "CompositionB": "5",
                    "Curve": "Initial",
                },
                {
                    "Time": "10",
                    "Flow": "1",
                    "CompositionA": "10",
                    "CompositionB": "90",
                    "Curve": "6",
                },
                {
                    "Time": "20",
                    "Flow": "1",
                    "CompositionA": "95",
                    "CompositionB": "5",
                    "Curve": "6",
                },
                {
                    "Time": "30",
                    "Flow": "1",
                    "CompositionA": "95",
                    "CompositionB": "5",
                    "Curve": "6",
                },
            ],
            method_name="test",
            column_temperature=30,
        )

        method = generate_ramp_method(
            method,
            low_flow_rate=0.05,
            flow_curve=6,
            ramp_type="rampdown",
            reduce_column_temperature=True,
        )
        assert method.gradient_table == [
            {
                "Time": "Initial",
                "Flow": "1",
                "CompositionA": "95",
                "CompositionB": "5",
                "Curve": "Initial",
            },
            {
                "Time": "1",
                "Flow": "0.05",
                "CompositionA": "95",
                "CompositionB": "5",
                "Curve": "6",
            },
        ]
        assert method.column_temperature == 20
        assert method.method_name == "test_low"

test_generate_ramp_method()

[{'Time': 'Initial', 'Flow': '0.05', 'CompositionA': '95', 'CompositionB': '5', 'Curve': 'Initial'}, {'Time': '10', 'Flow': '1', 'CompositionA': '95', 'CompositionB': '5', 'Curve': '6'}]
[{'Time': 'Initial', 'Flow': '0.05', 'CompositionA': '95', 'CompositionB': '5', 'Curve': 'Initial'}, {'Time': '10', 'Flow': '1', 'CompositionA': '95', 'CompositionB': '5', 'Curve': '6'}]
